# Przykład analizy na zaliczenie


Przedmiotem analizy powinien być trend lub zjawisko w sieci. Może to być analiza ruchu na konkretnej stronie www (dane z google analitics np. cvr.com.vn) lub dane z innego źródła (np. Google Trends).

Praca powinna zawierać:
- opis problemu
- źródło pozyskania danych
- wizualizacje 
- prognozę 
- wnioski (krótko)

Preferowana forma pracy: Notatnik Jupyter Nootebook lub plik Word / PDF

Prace należy przesłać mailem lub udstępnić na githubie marcinkieruzel

## Analiza ruchu na stronie www

Dane pochodzą ze Google Analitics strony cvr.com.vn 

Interesuje mnie analiza ruchu na stronie www i próba predykcji przyszłego ruchu uwzględniającego trend.

Używam modelu Facebook Prophet

In [ ]:
import pandas as pd

# Wczytanie danych z pliku CSV
data = pd.read_csv('all.csv')

print(data.head())

# Konwersja kolumny 'Date' na typ daty
data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d')

# Ustawienie kolumny 'Date' jako indeks
data.set_index('Date', inplace=True)

# Sortowanie danych po dacie
data.sort_index(inplace=True)

print(data.head())

In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt

# Przygotowanie danych dla Propheta (wymaga kolumn 'ds' i 'y')
df_prophet = data[['active_users_total']].reset_index()
df_prophet.columns = ['ds', 'y']

# Utworzenie i dopasowanie modelu z sezonowoscia dzienna
model = Prophet(daily_seasonality=True)
model.fit(df_prophet)

# Utworzenie ramki danych z przyszlymi datami (30 dni do przodu)
future = model.make_future_dataframe(periods=30)

# Generowanie prognozy
forecast = model.predict(future)

# Wykres prognozy
fig = model.plot(forecast)
plt.title('Aktywni uzytkownicy - Prognoza na kolejne 30 dni')
plt.xlabel('Data')
plt.ylabel('Liczba aktywnych uzytkownikow')
plt.tight_layout()
plt.show()

# Wykres skladowych modelu (trend, sezonowosc tygodniowa/dzienna)
fig2 = model.plot_components(forecast)
plt.tight_layout()
plt.show()

# Wyswietlenie prognozowanych wartosci na kolejny miesiac
print("Prognoza na kolejne 30 dni:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30).to_string(index=False))

## Wyjaśnienie analizy Prophet

**Facebook Prophet** to model szeregów czasowych opracowany przez Meta (Facebook), który automatycznie wykrywa:
- **Trend** – ogólny kierunek zmian danych w czasie (wzrost lub spadek),
- **Sezonowość tygodniową** – powtarzające się wzorce w cyklu tygodniowym (np. więcej użytkowników w określone dni tygodnia),
- **Sezonowość dzienną** – wahania w ciągu dnia.

### Jak czytać wykresy:
- **Wykres prognozy** – czarne punkty to dane rzeczywiste, niebieska linia to prognoza modelu, a jasnoniebieski obszar to **przedział ufności** (zakres, w którym z dużym prawdopodobieństwem znajdą się przyszłe wartości).
- **Wykres składowych** – rozbija prognozę na poszczególne komponenty: trend, sezonowość tygodniową i dzienną, co pozwala zrozumieć, co wpływa na liczbę użytkowników.

### Kolumny w tabeli prognozy:
| Kolumna | Znaczenie |
|---|---|
| `ds` | Data |
| `yhat` | Prognozowana wartość (liczba aktywnych użytkowników) |
| `yhat_lower` | Dolna granica przedziału ufności |
| `yhat_upper` | Górna granica przedziału ufności |

---

## Wnioski z analizy

### 1. Dane wejściowe
- Analiza obejmuje **62 dni** (od 4 grudnia 2025 do 3 lutego 2026).
- Średnia dzienna liczba aktywnych użytkowników wynosi **359**, przy czym wartości wahają się od 195 do 700.

### 2. Wyraźny trend wzrostowy
- Średnia z pierwszego tygodnia to **243** użytkowników, a z ostatniego **409** – co oznacza **wzrost o ok. 69%**.
- Model Prophet potwierdza ten rosnący trend, prognozując dalszy wzrost w kolejnym miesiącu.
- Widzimy jednak wyrażne outliery

### 3. Prognoza na kolejne 30 dni
- Prognozowana średnia liczba użytkowników to **ok. 532** dziennie.
- Wartości prognozowane wahają się od **430 do 634**, co sugeruje kontynuację trendu wzrostowego.

### 4. Dominacja ruchu mobilnego
- Użytkownicy mobilni stanowią **24,2%** zidentyfikowanego ruchu, a desktopowi jedynie **0,7%**.
- Pozostałe ~75% ruchu pochodzi z innych źródeł nieuwzględnionych w podziale mobile/desktop.

### 5. Zalecenia
- **Optymalizacja pod urządzenia mobilne** powinna być priorytetem ze względu na dominację tego kanału.
- Należy monitorować, czy trend wzrostowy się utrzyma – prognoza opiera się na stosunkowo krótkim okresie danych (62 dni), co może wpływać na jej dokładność.
- Warto rozszerzyć analizę o dłuższy okres danych, aby Prophet mógł lepiej wykryć sezonowość i dostarczyć bardziej wiarygodne prognozy.